# 4.1 Review Features: Data Preparation

## Helpers

In [79]:
import sys, time
_st = None
def timeit():
    global _st
    if _st is None:
        _st = time.time()
    else:
        print(int(time.time() - _st))
        _st = None

## Loading Data

In [74]:
from datetime import datetime as dt
import pandas as pd

In [76]:
DIR_PREP = '../data/prepared/'
PATH_REVIEWS = DIR_PREP + 'reviews/%d.csv'
PATH_LANGS = DIR_PREP + 'reviews/languages/%d.csv'
REVIEW_PAGES = 32
LANGS_PAGES = 21

In [146]:
def load_reviews():
    df = pd.concat((
        pd.read_csv(PATH_REVIEWS % page_num, header=None)
        for page_num in range(1, REVIEW_PAGES + 1)
    )).reset_index(drop=True)
    df.columns = [
        'uid','gid','polarity','ea','pt_forever','pt_review',
        'ts_created','ts_updated','votes_up','votes_funny','text'
    ]
    return df

def load_reviews_lang():
    df = pd.concat((
        pd.concat((
            pd.read_csv(PATH_REVIEWS % page_num, header=None),
            pd.read_csv(PATH_LANGS % page_num, header=None)
        ), axis=1)
        for page_num in range(1, LANGS_PAGES + 1)
    )).reset_index(drop=True)
    df.columns = [
        'uid','gid','polarity','ea','pt_forever','pt_review',
        'ts_created','ts_updated','votes_up','votes_funny','text',
        'lang','confidence','nlp_words','raw_words','raw_chars'
    ]
    return df

In [80]:
df_all = load_reviews()
df_lang = load_reviews_lang()

## Sample Data

In [82]:
DIR_SAMPLE = '../data/samples/'
PATH_REVSENTS = DIR_SAMPLE + 'review_sentiment/%s.csv'

In [235]:
def prepare_review_sentiment_data(df, N=10000, min_conf=0.7, long_words=50):
    is_valid = df['nlp_words'] >= 1
    is_english = (df['lang'] == 'en') & (df['confidence'] >= min_conf)
    is_long = df['nlp_words'] >= long_words
    # English only, equal polarities, any length
    df[is_valid & is_english].groupby('polarity').sample(n=(N // 2)).to_csv(PATH_REVSENTS % f'eng_eq_any_{N}', index=False)
    # English only, any polarity, any length
    df[is_valid & is_english].sample(n=N).to_csv(PATH_REVSENTS % f'eng_any_any_{N}', index=False)
    # English only, equal polarities, short
    df[is_valid & is_english & ~is_long].groupby('polarity').sample(n=(N // 2)).to_csv(PATH_REVSENTS % f'eng_eq_short_{N}', index=False)
    # English only, any polarity, short
    df[is_valid & is_english & ~is_long].sample(n=N).to_csv(PATH_REVSENTS % f'eng_any_short_{N}', index=False)
    # English only, equal polarities, long
    df[is_valid & is_english & is_long].groupby('polarity').sample(n=(N // 2)).to_csv(PATH_REVSENTS % f'eng_eq_long_{N}', index=False)
    # English only, any polarity, long
    df[is_valid & is_english & is_long].sample(n=N).to_csv(PATH_REVSENTS % f'eng_any_long_{N}', index=False)
    # Any language, equal polarities, any length
    df[is_valid].groupby('polarity').sample(n=(N // 2)).to_csv(PATH_REVSENTS % f'any_eq_any_{N}', index=False)
    # Any language, any polarity, any length
    df[is_valid].sample(n=N).to_csv(PATH_REVSENTS % f'any_any_any_{N}', index=False)
    # Any language, equal polarities, short
    df[is_valid & ~is_long].groupby('polarity').sample(n=(N // 2)).to_csv(PATH_REVSENTS % f'any_eq_short_{N}', index=False)
    # Any language, any polarity, short
    df[is_valid & ~is_long].sample(n=N).to_csv(PATH_REVSENTS % f'any_any_short_{N}', index=False)
    # Any language, equal polarities, long
    df[is_valid & is_long].groupby('polarity').sample(n=(N // 2)).to_csv(PATH_REVSENTS % f'any_eq_long_{N}', index=False)
    # Any language, any polarity, long
    df[is_valid & is_long].sample(n=N).to_csv(PATH_REVSENTS % f'any_any_long_{N}', index=False)

In [236]:
prepare_review_sentiment_data(df_lang, N=100000)